<a href="https://colab.research.google.com/github/kimsun75/dataAnalysis/blob/main/3_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%ED%9A%8C%EA%B7%80_2_%EC%95%84%ED%8C%8C%ED%8A%B8%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80%EC%98%88%EC%B8%A1_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] 아파트 실거래 가격 예측

- 서울/부산 지역의 아파트 관련 정보를 바탕으로 실거래가 예측
- 데이콘 대회 문제: 직방 거래 데이터(2016~2017)
- 참고: https://dacon.io/competitions/official/21265/overview/description

* 사용 데이터
    - 아파트 실거래가 데이터(아파트실거래가격.csv
    - 행정구역분류 데이터(한국행정구역분류.xlsx)
    - 공원 데이터(park.csv)
    - 어린이집 데이터(day_care_center.csv)
    - 테스트 데이터(아파트실거래가격_test.csv)
* 사용 모델
    - 앙상블 모델(RandomForestRegressor, xgboost, lightgbm)


### #그래프에서 한글사용하는 방법
- **(코랩에서)한글폰트 설치하기**

In [1]:
!pip install -U feature-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.3/344.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 43.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 코랩에서 위 코드를 실행시킨 후  반드시 코랩 메뉴: "런타임>세션 다시 시작" 합니다.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (38.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121918 files and direc

In [ ]:
# 코랩에서 한글 폰트 종류와 이름이 win과 다를 수 있다!!!
import matplotlib.pyplot as plt
plt.rcParams.update({'font.family': 'NanumGothic',
                     'font.size': 12,
                     'figure.figsize': (10, 5),
                     'axes.unicode_minus':  False }) # 폰트 설정



---



## 1.데이터 준비하기

In [2]:
# 데이터 가져오기
import pandas as pd
df = pd.read_csv("아파트실거래가격.csv", engine = "python", encoding = "utf8")
df.head(2)

FileNotFoundError: [Errno 2] No such file or directory: '아파트실거래가격.csv'

## 2.데이터 전처리

### 2-1. 기본 정보 확인

- 데이터 요약정보

In [ ]:
df.info()

- 데이터의 지역 확인(city: 서울특별시', '부산광역시)

In [ ]:
df.city.unique()

In [ ]:
df.city.value_counts()

- 거래 년도 확인(transaction_year_month: 201601~201711)

In [ ]:
df.transaction_year_month.unique()

In [ ]:
# 거래발생년월별 데이터 건수
data = df.transaction_year_month.value_counts()
data = data.sort_index()
data

- 특정 아파트 거래내역 확인

In [ ]:
df.query("apartment_id == 1878")

### 2-3.데이터 처리

#### 1) 필요 데이터 처리(파생변수 추가, 변수 삭제, 값 변경)


#### #행정구역(시군구) 변수 추가

In [ ]:
# 참조 데이터 불러오기
ref_df = pd.read_excel("한국행정구역분류.xlsx",
                       sheet_name = "법정동코드 연계 자료분석용",
                       header = 1)

ref_df.head()

In [ ]:
# ref_df 필터링
ref_df = ref_df.loc[ref_df['시도'].isin(['서울특별시', '부산광역시'])] # 서울특별시와 부산광역시 레코드만 가져옴
ref_df = ref_df[['시도', '시군구', '법정동']]
ref_df.head()

In [ ]:
# ref_df에 포함되어 있는 시도-읍면동과 df에 포함되어 있는 시도-읍면동이 일치하지 않는 경우를 확인해야 함
ref_df_loc = ref_df['시도'] + '-' + ref_df['법정동']
df_loc = (df['city'] + '-' + df['dong']).unique()

import numpy as np
np.isin(df_loc, ref_df_loc) # df_loc가 ref_df_loc에 포함되지 않는 경우가 있음을 확인

# 포함되지 않는 동 목록 확인: 전부 부산광역시이며 읍 단위인 경우에 이러한 문제가 발생함을 확인
df_loc[~np.isin(df_loc, ref_df_loc)]

In [ ]:
# 장안읍 예시:
ref_df.loc[ref_df['법정동'].str.contains('장안')]

In [ ]:
# 시도와 법정동이 완전히 똑같은 행이 있어, 이를 제거함
ref_df = ref_df.drop_duplicates(subset = ['시도', '법정동'])

In [ ]:
# 결론: df의 dong에 리가 붙어있으면 제거해야 함
df['dong'] = df['dong'].str.split(' ', expand = True).iloc[:, 0]

# 재확인: 정상적으로 작동함을 확인
df_loc = (df['city'] + '-' + df['dong']).unique()
df_loc[~np.isin(df_loc, ref_df_loc)]

In [ ]:
# df와 ref_df 병합
df = pd.merge(df, ref_df, left_on = ['city', 'dong'], right_on = ['시도', '법정동'])

# ref_df에 있던 불필요한 변수 제거
df.drop(['시도', '법정동'], axis = 1, inplace = True)
df.head(2)

#### #불필요한 변수 제거

In [ ]:
df.drop(['transaction_id', 'addr_kr'], axis = 1, inplace = True) # transaction_id는 인덱스와 동일하므로 삭제해도 무방

In [ ]:
# apartment_id는 id지만 어느정도 사용이 가능할 것이라 보임 (완전히 유니크하지 않으므로)
df['apartment_id'].value_counts().sort_values(ascending=False).head(10)

In [ ]:
# 거래가 데이터가 많은 아파트
df.query("apartment_id == 10939").head(2)

#### #파생변수 생성
- age : 건물 나이(2018년도 기준)
- transaction_year : 거래 년
- transaction_month: 거래 월
- Seoul : 서울 여부
- floor_level : 층(floor)별 가격 구간

In [ ]:
# 건축된지 얼마나 되었는지를 나타내는 변수로 변환
# 최근거래일자가 201711이므로 2018년을 기준으로 한다.
df['age'] = 2018 - df['year_of_completion']
df.drop('year_of_completion', axis = 1, inplace = True)

In [ ]:
# 거래 년도 추출
# str accessor를 사용하기 위해, 타입 변경
df['transaction_year_month'] = df['transaction_year_month'].astype(str)

df['transaction_year'] = df['transaction_year_month'].str[:4].astype(int)
df['transaction_month'] = df['transaction_year_month'].str[4:].astype(int)
df.drop('transaction_year_month', axis = 1, inplace = True)

In [ ]:
# 도시가 둘 뿐이므로 서울특별시인지 나타내는 이진 변수 생성
df['Seoul'] = (df['city'] == "서울특별시").astype(int)

In [ ]:
# floor 구간화 필요: 2층과 3층의 가격 차이가 있을까?
# 대략적인 구간 확인: 1층 이하, 1 ~ 25층, 26층 ~ 49층, 50층 이상
df.groupby(['floor'])['transaction_real_price'].mean().plot(kind = 'bar')

In [ ]:
# 층 수준을 생성한 뒤, 차이를 봄
def floor_level_converter(x):
    if x <= 15:
        return "low"
    elif x <= 25:
        return "middle"
    elif x <= 51:
        return "high"
    else:
        return "very_high"

df['floor_level'] = df['floor'].apply(floor_level_converter)

In [ ]:
# 저층 그룹에 아파트의 층에 따른 가격 분포 확인 => 층에 따른 차이가 크지 않음
df.loc[df['floor_level'] == 'low'].boxplot(column = 'transaction_real_price', by = 'floor')

In [ ]:
# 중층 그룹에 아파트의 층에 따른 가격 분포 확인 => 층에 따른 차이가 크지 않음
df.loc[df['floor_level'] == 'middle'].boxplot(column = 'transaction_real_price', by = 'floor')

In [ ]:
# 고층 그룹에 아파트의 층에 따른 가격 분포 확인 => 층에 따른 차이가 크지 않음
df.loc[df['floor_level'] == 'high'].boxplot(column = 'transaction_real_price', by = 'floor')

In [ ]:
# 초고층 그룹에 아파트의 층에 따른 가격 분포 확인 => 층이 높으면 높을수록 가격이 더 오름 (단, 72층을 기준으로 그런 현상이 두드러짐)
df.loc[df['floor_level'] == 'very_high'].boxplot(column = 'transaction_real_price', by = 'floor')

# 결론: floor_level 변수는 유의하나, floor 변수는 초고층에서만 유의
# 따라서 very high를 세분화하거나, floor_level 변수와 floor 변수를 같이 사용하고 교호작용을 고려할 수 있는 트리 계열의 모델 사용이 적합
# 여기서는 very high를 세분화하기로 결정

In [ ]:
# 층 수준을 생성한 뒤, 차이를 봄
def floor_level_converter(x):
    if x <= 15:
        return "low"
    elif x <= 25:
        return "middle"
    elif x <= 51:
        return "high"
    elif x <= 71: # 새로운 조건 추가
        return "very_high"
    else:
        return "very_very_high"

df['floor_level'] = df['floor'].apply(floor_level_converter)
df.drop('floor', axis = 1, inplace = True)

#### 시세 변수 추가

In [ ]:
# 구별 전체 평균 시세 부착
mean_price_per_gu = df.groupby(['city', '시군구'], as_index = False)['transaction_real_price'].mean()
mean_price_per_gu.rename({'transaction_real_price':'구별_전체_평균_시세'}, axis = 1, inplace = True)
df = pd.merge(df, mean_price_per_gu, on = ['city', '시군구'])
df.head(2)

In [ ]:
# 구별 작년 시세 부착
# price_per_gu_and_year 변수에 직접 수정을 하므로, df가 변경되는 것을 방지하기 위해, df.copy().groupby~를 사용
price_per_gu_and_year = df.copy().groupby(['city', '시군구', 'transaction_year'], as_index = False)['transaction_real_price'].agg(['mean', 'count'])
price_per_gu_and_year = price_per_gu_and_year.reset_index().rename({"mean":"구별_작년_평균_시세", "count":"구별_작년_거래량"}, axis = 1)

price_per_gu_and_year['transaction_year'] += 1 # 작년것을 붙이기 위해, 1을 더함
df = pd.merge(df, price_per_gu_and_year, on = ['city', '시군구', 'transaction_year'], how = 'left') # 작년 기록이 없어서 붙지 않는 것을 방지하기 위해, how = 'left'로 설정
df['구별_작년_거래량'].fillna(0, inplace = True) # 구별 작년 거래 데이터가 없다는 것은, 구별 작년 거래량이 0이라는 이야기이므로 fillna(0)을 수행

In [ ]:
# 아파트별 평균 시세 부착
price_per_aid = df.copy().groupby(['apartment_id'], as_index = False)['transaction_real_price'].mean()
price_per_aid.rename({"transaction_real_price":"아파트별_평균가격"}, axis = 1, inplace = True)

df = pd.merge(df, price_per_aid, on = ['apartment_id'])

In [ ]:
df.head(2)

#### 2) 외부 데이터 부착

#### #공원 데이터

In [ ]:
park_df = pd.read_csv("park.csv")
park_df.head()

In [ ]:
# 모든 값을 고려하는 것은 불가능하고, 그리 좋은 접근이 아닌 것으로 보임
park_df['park_exercise_facility'].unique()

In [ ]:
park_df.columns

In [ ]:
# 따라서 결측을 0으로, 결측이 아니면 1로 변환
facility_cols = ['park_exercise_facility', 'park_entertainment_facility', 'park_benefit_facility', 'park_cultural_facitiy', 'park_facility_other']
for col in facility_cols:
    park_df.loc[park_df[col].notnull(), col] = 1
    park_df.loc[park_df[col].isnull(), col] = 0

In [ ]:
# 동별 공원 수
num_park_per_dong = park_df.groupby(['city', 'gu', 'dong'], as_index = False)['park_name'].count()
num_park_per_dong.rename({"park_name":"공원수"}, axis = 1, inplace = True)

# 동별 공원에 배치된 facilty 수
num_facilty_per_dong = park_df.groupby(['city', 'gu', 'dong'], as_index = False)[facility_cols].sum()
num_facilty_per_dong.head()

In [ ]:
# 공원 데이터 부착
df = pd.merge(df, num_park_per_dong, left_on = ['city', '시군구', 'dong'], right_on = ['city', 'gu', 'dong'], how = 'left')
df['공원수'].fillna(0, inplace = True)
df.drop('gu', axis = 1, inplace = True)

df = pd.merge(df, num_facilty_per_dong, left_on = ['city', '시군구', 'dong'], right_on = ['city', 'gu', 'dong'], how = 'left')
df[facility_cols].fillna(0, inplace = True)
df.drop('gu', axis = 1, inplace = True)

#### #어린이집 데이터 부착

In [ ]:
day_care_center_df = pd.read_csv('day_care_center.csv')
day_care_center_df.head()

# 가설: 같은 어린이집이어도 종류가 다르면 아파트 가격에 다르게 영향을 줄 것이다. 가령, 가정 어린이집보단 국공립어린이집이 더 인기가 좋을 것이다.
# 또한, 아이가 있는 부모라면, 어린이집 수와 케어 가능한 아이의 수 등만 보고 아파트 구매를 결정하지, 각 어린이집에 CCTV 개수가 몇개인지 등까진 파악하지 않을 것이다.
# 따라서 구 및 유형별 어린이집 수와 케어 가능한 아이 수만 집계

In [ ]:
# 불필요한 변수 제거
day_care_center_df = day_care_center_df[['city', 'gu', 'day_care_type', 'day_care_baby_num']]

In [ ]:
dummy_day_care_type = pd.get_dummies(day_care_center_df['day_care_type'], drop_first = False)
dummy_day_care_type = dummy_day_care_type.add_prefix("어린이집유형_")

day_care_center_df = pd.concat([day_care_center_df, dummy_day_care_type], axis = 1)
day_care_center_df.drop('day_care_type', axis = 1, inplace = True)
day_care_center_df.head()

In [ ]:
aggregated_day_care_center_df = day_care_center_df.groupby(['city', 'gu'], as_index = False)[day_care_center_df.columns[2:]].sum()
aggregated_day_care_center_df.head()

In [ ]:
# 어린이집 데이터 부착
df = pd.merge(df, aggregated_day_care_center_df, left_on = ['city', '시군구'], right_on = ['city', 'gu'], how = 'left')
df[aggregated_day_care_center_df.columns].fillna(0, inplace = True)
df.drop('gu', axis = 1, inplace = True)

In [ ]:
df.head(2)

## 3.모델링

### 3-1. 학습용 데이터 준비

In [ ]:
X = df.drop(['apartment_id', 'city', 'dong', 'jibun', 'apt', 'transaction_date', 'transaction_real_price', '시군구', 'transaction_year', 'transaction_month'], axis = 1)
Y = df['transaction_real_price']
X.head()
X.columns

### 3-2. 학습용, 테스트 데이터 분리하기

In [ ]:
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)
Train_X.shape # 샘플 대비 특징이 매우 적음 => 큰 부담없이 더미화 가능
# 컬럼 타입이 섞여 있으므로, 트리 계열의 모델이 적절한 것으로 판단됨
# 또한, 샘플이 충분히 많으므로 트리 뿐만 아니라 트리 기반의 앙상블도 적절할 것으로 보임
# Tip. Decision Tree 계열의 모델은 상대적으로 전처리나 탐색의 부담이 제일 적은 모델임

#### #인코딩

In [ ]:
# 더미화
# from feature_engine.categorical_encoders import OneHotCategoricalEncoder as OHE
from feature_engine.encoding import OneHotEncoder as OHE

dummy_model = OHE(variables = ['floor_level'],
                 drop_last = False)

dummy_model.fit(Train_X)

Train_X = dummy_model.transform(Train_X)
Test_X = dummy_model.transform(Test_X)

In [ ]:
# 변수 부착 과정에서 생성된 결측 대체
from sklearn.impute import SimpleImputer as SI
imputer = SI().fit(Train_X)
Train_X = pd.DataFrame(imputer.transform(Train_X), columns = Train_X.columns)
Test_X = pd.DataFrame(imputer.transform(Test_X), columns = Test_X.columns)

### 3-3.모델 훈련

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.feature_selection import *
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGB
from lightgbm import LGBMRegressor as LGB

In [ ]:
model_parameter_dict = dict()
RFR_parameter_grid = ParameterGrid({"max_depth":[3, 4, 5],
                                   "n_estimators": [100, 200]})

XL_parameter_grid = ParameterGrid({"max_depth":[3, 4, 5],
                                  "n_estimators": [100, 200],
                                  "learning_rate": [0.05, 0.1, 0.2]})

model_parameter_dict[RFR] = RFR_parameter_grid
model_parameter_dict[XGB] = XL_parameter_grid
model_parameter_dict[LGB] = XL_parameter_grid

In [ ]:
# 출력을 위한 max_iter_num 계산
max_iter_num = len(range(20, 4, -5)) * len(model_parameter_dict) * len(XL_parameter_grid) ** 2

from sklearn.metrics import mean_absolute_error as MAE
best_score = 9999999999
iteration_num = 0
for k in range(20, 4, -5):
    selector = SelectKBest(mutual_info_regression, k = k).fit(Train_X, Train_Y)
    s_Train_X = selector.transform(Train_X)
    s_Test_X = selector.transform(Test_X)


    for model_func in model_parameter_dict.keys():
        for parameter in model_parameter_dict[model_func]:
            model = model_func(**parameter).fit(s_Train_X, Train_Y) # Light GBM은 DataFrame의 컬럼 이름에 대한 제약이 있어, ndarray를 입력으로 사용
            pred_Y = model.predict(s_Test_X)
            score = MAE(Test_Y, pred_Y)

            if score < best_score:
                print(k, model_func, parameter, score)
                best_score = score
                best_model_func = model_func
                best_parameter = parameter
                best_selector = selector

            iteration_num += 1
            print("iter_num:{}/{}, score: {}, best_score: {}".format(iteration_num, max_iter_num, round(score, 2), round(best_score, 2)))

In [ ]:
final_X = pd.concat([Train_X, Test_X], axis = 0, ignore_index = True)
final_Y = pd.concat([Train_Y, Test_Y], axis = 0, ignore_index = True)

final_model = best_model_func(**best_parameter).fit(best_selector.transform(final_X), final_Y)

### 파이프라인 구축

In [ ]:
def pipeline(new_data, ref_df, model, selector, mean_price_per_gu, num_park_per_dong, num_facilty_per_dong, aggregated_day_care_center_df, imputer, dummy_model):
    ## 변수 변환 및 부착
    new_data['dong'] = new_data['dong'].str.split(' ', expand = True).iloc[:, 0] # dong에 리가 붙어있으면 제거

    new_data = pd.merge(new_data, ref_df, left_on = ['city', 'dong'], right_on = ['시도', '법정동']) # 시군구 부착

    new_data.drop(['시도', '법정동', 'transaction_id', 'addr_kr'], axis = 1, inplace = True) # 불필요한 변수 제거

    # age 변수 부착
    new_data['age'] = 2018 - new_data['year_of_completion']
    new_data.drop('year_of_completion', axis = 1, inplace = True)

    # 거래 년월 부착
    new_data['transaction_year_month'] = new_data['transaction_year_month'].astype(str)
    new_data['transaction_year'] = new_data['transaction_year_month'].str[:4].astype(int)
    new_data['transaction_month'] = new_data['transaction_year_month'].str[4:].astype(int)
    new_data.drop('transaction_year_month', axis = 1, inplace = True)

    # Seoul 생성
    new_data['Seoul'] = (new_data['city'] == "서울특별시").astype(int)

    # floor_level 변수 생성
    new_data['floor_level'] = new_data['floor'].apply(floor_level_converter)
    new_data.drop('floor', axis = 1, inplace = True)

    # 시세 관련 변수 추가
    new_data = pd.merge(new_data, mean_price_per_gu, on = ['city', '시군구'])
    new_data = pd.merge(new_data, price_per_gu_and_year, on = ['city', '시군구', 'transaction_year'], how = 'left')
    new_data['구별_작년_거래량'].fillna(0, inplace = True) # 구별 작년 거래 데이터가 없다는 것은, 구별 작년 거래량이 0이라는 이야기이므로 fillna(0)을 수행

    new_data = pd.merge(new_data, price_per_aid, on = ['apartment_id'], how = 'left')


    # 공원 데이터 부착
    new_data = pd.merge(new_data, num_park_per_dong, left_on = ['city', '시군구', 'dong'], right_on = ['city', 'gu', 'dong'], how = 'left')
    new_data['공원수'].fillna(0, inplace = True)
    new_data.drop('gu', axis = 1, inplace = True)

    new_data = pd.merge(new_data, num_facilty_per_dong, left_on = ['city', '시군구', 'dong'], right_on = ['city', 'gu', 'dong'], how = 'left')

    facility_cols = ['park_exercise_facility', 'park_entertainment_facility', 'park_benefit_facility', 'park_cultural_facitiy', 'park_facility_other']
    new_data[facility_cols].fillna(0, inplace = True)
    new_data.drop('gu', axis = 1, inplace = True)

    # 어린이집 데이터 부착
    new_data = pd.merge(new_data, aggregated_day_care_center_df, left_on = ['city', '시군구'], right_on = ['city', 'gu'], how = 'left')
    new_data[aggregated_day_care_center_df.columns].fillna(0, inplace = True)
    new_data.drop('gu', axis = 1, inplace = True)

    # 특징 추출 ('transaction_real_price'는 drop 대상에서 제외)
    X = new_data.drop(['apartment_id', 'city', 'dong', 'jibun', 'apt', 'transaction_date', '시군구', 'transaction_year', 'transaction_month'], axis = 1)

    # 더미화
    X = dummy_model.transform(X)

    # 결측 대체
    X = imputer.transform(X)

    # 특징 선택
    X = selector.transform(X)

    return model.predict(X)

In [ ]:
import pickle
pipeline_element = {"ref_df": ref_df,
                   "model":final_model,
                   "selector":best_selector,
                   "mean_price_per_gu":mean_price_per_gu,
                   "num_park_per_dong":num_park_per_dong,
                   "num_facilty_per_dong":num_facilty_per_dong,
                   "aggregated_day_care_center_df":aggregated_day_care_center_df,
                   "imputer":imputer,
                   "dummy_model":dummy_model,
                   "pipeline":pipeline}

with open("아파트실거래가예측모델.pckl", "wb") as f:
    pickle.dump(pipeline_element, f)

In [ ]:
with open("아파트실거래가예측모델.pckl", "rb") as f:
    pipeline_element = pickle.load(f)

ref_df = pipeline_element["ref_df"]
model = pipeline_element["model"]
selector = pipeline_element["selector"]
mean_price_per_gu = pipeline_element["mean_price_per_gu"]
num_park_per_dong = pipeline_element["num_park_per_dong"]
num_facilty_per_dong = pipeline_element["num_facilty_per_dong"]
aggregated_day_care_center_df = pipeline_element["aggregated_day_care_center_df"]
imputer = pipeline_element["imputer"]
dummy_model = pipeline_element["dummy_model"]
pipeline = pipeline_element["pipeline"]

### 3-4.모델 테스트

In [ ]:
test_df = pd.read_csv("아파트실거래가격_test.csv")
test_df.head()

In [ ]:
output = pipeline(test_df, ref_df, model, selector, mean_price_per_gu, num_park_per_dong, num_facilty_per_dong, aggregated_day_care_center_df, imputer, dummy_model)
result = pd.Series(output, index = test_df['transaction_id'])
result